# GPUs

View the graphics card information

In [1]:
!nvidia-smi

Sat Nov 12 20:46:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   54C    P0    41W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


|   1  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   39C    P0    59W / 300W |   3376MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2...  Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   55C    P0    52W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   3  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   41C    P0    54W / 300W |   3284MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                        

Computing Devices

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

Query the number of available GPUs

In [3]:
torch.cuda.device_count()

2

Define two convenient functions that allow us
to run code even if the requested GPUs do not exist

In [4]:
def try_gpu(i=0):  
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

Query the device where the tensor is located

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

Store a tensor on the GPU

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

Create a random tensor on the second GPU

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.4144, 0.7294, 0.5390],
        [0.1520, 0.7057, 0.2716]], device='cuda:1')

If we want to compute `X + Y`,
we need to decide where to perform this operation

In [8]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


The data are on the same GPU
(both `Z` and `Y` are),
we can add them up

In [9]:
Y + Z

tensor([[1.4144, 1.7294, 1.5390],
        [1.1520, 1.7057, 1.2716]], device='cuda:1')

In [10]:
Z.cuda(1) is Z

True

Neural Networks and GPUs

In [12]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.2492],
        [-0.2492]], device='cuda:0', grad_fn=<AddmmBackward0>)

Confirm that the model parameters are stored on the same GPU

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)